In [ ]:
import os
import sys

os.environ

In [2]:
def get_attributes(object):
    attributes = dir(object)
    filtered_attributes = [
        attr for attr in attributes if not attr.startswith('__')]
    return filtered_attributes


def get_methods(object):
    methods = dir(object)
    filtered_methods = [
        method for method in methods if not method.startswith('__') and callable(getattr(object, method))]
    return filtered_methods


def get_properties(object):
    properties = dir(object)
    filtered_properties = [
        prop for prop in properties if not prop.startswith('__') and not callable(getattr(object, prop))]
    return filtered_properties

In [2]:
import requests
import json
import pandas as pd
from pprint import pprint
import pickle

base_url = "https://api.openalex.org/"
email = "katz.562@osu.edu"


def pickle_dump(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)


author_list = []
for j in range(10):
    try:
        author_list.extend(pickle.load(
            open(f"data/author_list_{j}.pkl", 'rb')))
        print(f"Author list {j} loaded from file.")
    except FileNotFoundError:
        sample = 10000
        per_page = 200
        while j < 10:
            new_author_list = []
            print(f"{j}th sample")
            seed = 3142+j
            i = 1
            while i <= sample/per_page:
                print(f"Page {i}")
                response = requests.get(base_url + "authors",
                                        params={'sample': sample, 'seed': seed, 'per-page': per_page, 'page': i})
                data = response.json()
                if len(data['results']) == 0:
                    break
                new_author_list.extend(data['results'])
                i += 1
            filename = f"data/author_list_{j}.json"
            print(f'Saving {len(new_author_list)} authors to {filename}')
            pickle_dump(new_author_list, f"data/author_list_{j}.pkl")
            j += 1

Author list 0 loaded from file.
Author list 1 loaded from file.
Author list 2 loaded from file.
Author list 3 loaded from file.
Author list 4 loaded from file.
Author list 5 loaded from file.
Author list 6 loaded from file.
Author list 7 loaded from file.
Author list 8 loaded from file.
Author list 9 loaded from file.


In [3]:
authors_df = pd.DataFrame(author_list)

In [4]:

# unnecessary_columns = ['orcid', 'ids', 'affiliations', 'last_known_institutions',
#    'summary_stats', 'created_date', 'updated_date', 'topics', 'topic_share', 'relevance_score']
df = pd.DataFrame(authors_df['display_name'])

In [5]:
def fetch_works(api_url):
    params = {
        'per_page': 200
    }
    response = requests.get(api_url, params)
    if response.status_code == 200:
        return [work['id'] for work in response.json()['results']]
    else:
        return None


n = 10  # Number of rows to process
df.loc[:n-1, 'works'] = df.loc[:n-1, 'works_api_url'].apply(fetch_works)

In [35]:
import regex as re


def get_first_name(display_name):
    first_name = r'([A-Z][a-z]+)'
    match = re.search(first_name, display_name)
    if match:
        return match.group(1)
    else:
        return None


def get_surname(display_name):
    match = re.search(r'\s([A-Z][a-z]+)$', display_name)
    if match:
        return match.group(1)
    else:
        return None


df['surname'] = df['display_name'].apply(get_surname)
df['first_name'] = df['display_name'].apply(get_first_name)

In [43]:
# import regex as re

# # Define the regex pattern
# pattern = r'^(?!.*\b(?:[A-Z]\.?\s?){1,3}(?:[a-z]+\s)*[A-Z][\p{L}]*\b).+$'


# # Filter the DataFrame using regex with a lambda function
# filtered_df = df[df['display_name'].apply(
#     lambda x: re.match(pattern, x) is not None)]

# # Display the filtered DataFrame
# filtered_df.head(100)

In [5]:
import re
df['strip_display_name'] = df['display_name'].str.replace(
    '\.\s?', ' ', regex=True)

# Filter the DataFrame
filtered_df = df[~df['strip_display_name'].str.match(r'^\b\w{1,2}\b ')]

# Split 'strip_display_name' into 'first' and 'last' using .loc to avoid SettingWithCopyWarning
filtered_df.loc[:,
                'first'] = filtered_df['strip_display_name'].str.split().str[0]
filtered_df.loc[:,
                'last'] = filtered_df['strip_display_name'].str.split().str[-1]
filtered_df.head(100)

/var/folders/ph/vmwntg591nz4p53wxs1w08dr0000gn/T/ipykernel_68138/3982131095.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.loc[:,
/var/folders/ph/vmwntg591nz4p53wxs1w08dr0000gn/T/ipykernel_68138/3982131095.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.loc[:,


,display_name,strip_display_name,first,last
0,Resty Fastabikul Khaerat,Resty Fastabikul Khaerat,Resty,Khaerat
2,Xinshun Li,Xinshun Li,Xinshun,Li
8,堀越さな恵,堀越さな恵,堀越さな恵,堀越さな恵
9,Debbie Northin,Debbie Northin,Debbie,Northin
10,Ambrish,Ambrish,Ambrish,Ambrish
...,...,...,...,...
138,付权茂,付权茂,付权茂,付权茂
139,Jawharah Fahad Abdul Aziz Al-Fehaid,Jawharah Fahad Abdul Aziz Al-Fehaid,Jawharah,Al-Fehaid
140,Steven Tschantzk,Steven Tschantzk,Steven,Tschantzk
143,Ερμής Γκίνης,Ερμής Γκίνης,Ερμής,Γκίνης


In [34]:
# from chicksexer import predict_gender
import gender_ai as g
import gender_guesser.detector as gender_guesser
d = gender_guesser.Detector()


gdf = filtered_df.copy().head(100)
gdf['gender_guesser'] = gdf['first'].apply(lambda x: d.get_gender(x))
gdf['gender_ai'] = gdf['first'].apply(lambda x: g.predict(x))
# gdf['chicksexer'] = gdf['first'].apply(lambda x: predict_gender(x))

/Users/jukatz/miniconda3/lib/python3.11/site-packages/gender_ai/__init__.py:123: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))


In [ ]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


def get_gender_prediction(input_string):
    response = client.responses.create(
        model='gpt-3.5-turbo',
        instructions='You are a helpful assistant that determines the most likely gender of a person with specified name: responding only with "Male", "Female", or "Unknown".',
        input=f"{input_string}."
    )
    # Extract the response text
    gender = response.output_text
    return gender

In [44]:
gdf['gpt2'] = gdf['display_name'].apply(lambda x: get_gender_prediction(x))

In [45]:
gdf

,display_name,strip_display_name,first,last,gender_guesser,gender_ai,gpt,gpt2
0,Resty Fastabikul Khaerat,Resty Fastabikul Khaerat,Resty,Khaerat,unknown,Girl,Unknown,Unknown
2,Xinshun Li,Xinshun Li,Xinshun,Li,unknown,probably a Boy,Male,Male
8,堀越さな恵,堀越さな恵,堀越さな恵,堀越さな恵,unknown,Boy,Female,Female
9,Debbie Northin,Debbie Northin,Debbie,Northin,female,Girl,Female,Female
10,Ambrish,Ambrish,Ambrish,Ambrish,unknown,Girl,Male,Male
...,...,...,...,...,...,...,...,...
138,付权茂,付权茂,付权茂,付权茂,unknown,Girl,Unknown,Male
139,Jawharah Fahad Abdul Aziz Al-Fehaid,Jawharah Fahad Abdul Aziz Al-Fehaid,Jawharah,Al-Fehaid,unknown,Boy,Unknown,Female
140,Steven Tschantzk,Steven Tschantzk,Steven,Tschantzk,male,Boy,Male,Male
143,Ερμής Γκίνης,Ερμής Γκίνης,Ερμής,Γκίνης,unknown,Girl,Male,Unknown
